In [ ]:
import folium
from owslib.wms import WebMapService
from owslib.wmts import WebMapTileService

# WMS example

In [ ]:
# issue with proxy/vpn, so do without
# wms = WebMapService('https://neo.gsfc.nasa.gov/wms/wms', version='1.1.1') # doesn't seem to work with folium?
# wms = WebMapService('https://img.nj.gov/imagerywms/Natural2015', version='1.1.1') # this works!
# can we swap WMS and WMTS?
wms = WebMapTileService('https://maps.nj.gov/arcgis/rest/services/Basemap/Orthos_Natural_2015_NJ/MapServer/WMTS', version='1.0.0')

In [ ]:
wms.url

In [ ]:
wms.identification.type

In [ ]:
wms.identification.version

In [ ]:
wms.identification.title

In [ ]:
wms.identification.abstract

In [ ]:
layer_ids = sorted(wms.contents)
len(layer_ids)

In [ ]:
layer_id = layer_ids[0]
layer_id

In [ ]:
layer = wms.contents[layer_id]

In [ ]:
layer.styles

In [ ]:
# not in WMTS!
layer.crsOptions

In [ ]:
layer.title

In [ ]:
# appears to be list of BoundingBox objects for WMTS vice 5-tuple for WMS
layer.boundingBox

In [ ]:
sorted(layer.styles.keys())[0:1]

In [ ]:
# for WMS
mp = wms.getmap(
    layers=[layer_id],
    # styles=sorted(layer.styles.keys())[0:1],#[layer.styles['rgb']],
    srs=layer.boundingBox[-1], #layer.crsOptions[0],
    bbox=layer.boundingBox[:-1], #(-70.8, 42, -70, 42.8),
    format='image/png',
    size=(256, 256),
    time=None,
    transparent=True,
)

In [ ]:
# for WMTS; not quite working (for just nj???)?
mp = wms.gettile(
    layer=layer_id,
    style=sorted(layer.styles.keys())[0:1],
    format='image/png',
    tilematrixset='default028mm',
    tilematrix='0',
    row=0,
    column=0,
)

In [ ]:
mp.geturl()

In [ ]:
mp.info()

In [ ]:
with open('nasa.png', 'wb') as fh:
    fh.write(mp.read())

In [ ]:
# Should be able to open png and see upside down? thumbnail of earth

## Visualize using folium

In [ ]:
# example of folium map pulled from OSM, overlaid with rain fall/reflectivity?
# dates include Hurricane Sandy's landfall in NJ, as well as same year's Derecho's in NJ
m = folium.Map(
    # tiles=None, # this still works, i.e., overlay below appears!
    tiles='OpenStreetMap',
)
l = folium.WmsTileLayer(
    url=wms.url,
    layers=[layer_id],
    # styles=sorted(layer.styles.keys())[0:1],#[layer.styles['rgb']],
    #srs=layer.boundingBox[-1], #layer.crsOptions[0],
    # bbox=layer.boundingBox[:-1], #(-70.8, 42, -70, 42.8),
    fmt='image/png',
    # size=(256, 256),
    # time=None,
    transparent=True,
    # attr='NASA...',
)
l.add_to(m, 'nasa')
m

In [ ]:
m.save('folium.html')